# Tied-Arch Bridge Optimization

## Libraries

In [ ]:
# Import the appropriate libraries

import math
from math import sqrt

## Constants

In [ ]:
# Define all the constants of the problem

fyk = 355000  # [kN/m^2] --- Nominal yield strength of steel S355
E = 2.1e8  # [kN/m^2] --- Young's Modulus
L = 65  # [m] --- Span length
Ai = 0.618  # [m^2] --- Cross-sectional area of the deck
W = 0.656  # [m^3] --- Section modulus of the deck

rho = 7850  # [kg/m^3] --- Steel density
GWP = 2.044  # [-] --- Global Warming Potential for steel S355
costS355 = 0.72  # [€/kg] --- Cost per kg of steel S355

## Dictionary for manual values

In [ ]:
# Define a dictionary that contains, for each row, all the values to be manually adjusted for each design scenario analyzed
# Specifically, the first row contains the data for the original case study; subsequent rows contain the data for alternative design scenarios
# Whenever a new design scenario needs to be added, simply add a new row to this dictionary, filling in all the indicated fields (Aa, f, Iy, Ix, etc.)

studies = {
    'original': {'Aa': 0.163, 'f': 6.89, 'Iy': 2.32e-2, 'Ix': 7.69e-3, 'N_Eda': 12498, 'N_Edi': 11810, 'V_Edi': 2000, 'M_Edi': 11911, 'alpha_cr': 9.11},
    'case 1': {'Aa': 0.169, 'f': 2, 'Iy': 3.21e-2, 'Ix': 5.13e-3, 'N_Eda': 22458, 'N_Edi': 22348, 'V_Edi': 2743, 'M_Edi': 46540, 'alpha_cr': 8.07},
    'case 2': {'Aa': 0.161, 'f': 4, 'Iy': 2.65e-2, 'Ix': 4.82e-3, 'N_Eda': 18140, 'N_Edi': 17810, 'V_Edi': 2562, 'M_Edi': 20373, 'alpha_cr': 8.53},
    'case 3': {'Aa': 0.153, 'f': 6, 'Iy': 2.52e-2, 'Ix': 3.32e-3, 'N_Eda': 13880, 'N_Edi': 13338, 'V_Edi': 2214, 'M_Edi': 14466, 'alpha_cr': 10.03},
    'case 4': {'Aa': 0.156, 'f': 8, 'Iy': 2.14e-2, 'Ix': 3.66e-3, 'N_Eda': 11273, 'N_Edi': 10525, 'V_Edi': 2063, 'M_Edi': 12033, 'alpha_cr': 10.08},
    'case 5': {'Aa': 0.156, 'f': 10, 'Iy': 2.94e-2, 'Ix': 3.89e-3, 'N_Eda': 9547, 'N_Edi': 8609, 'V_Edi': 1996, 'M_Edi': 11253, 'alpha_cr': 10.06},
}

## Dependent variables

In [ ]:
# In this section, I use some of the values in the dictionary 'studies' to calculate dependent variables theta, r, C, S

def calculate_S(case_data):
    Aa = case_data.get('Aa', None)
    f = case_data.get('f', None)

    if Aa is not None and f is not None:
        # Calculate dependent variables
        theta = (f * 48) / 6.89  # [°]
        r = (L / 2) * (1 / math.sin(math.radians(theta / 2)))  # [m]
        C = 2 * math.pi * r  # [m]
        S = (C * theta) / 360  # [m]

        CO2eqe = rho * Aa * GWP * S  # [kg]

        return theta, r, C, S
    else:
        return None

# Print the values of dependent variables theta, r, C, S
for case, study_data in studies.items():
    result = calculate_S(study_data)
    if result is not None:
        theta, r, C, S = result
        # Round the values to two decimal places and print in bold only the numbers
        theta_str = "{:.2f}".format(theta)
        r_str = "{:.2f}".format(r)
        C_str = "{:.2f}".format(C)
        S_str = "{:.2f}".format(S)
        print(f'{case}: theta = \033[1m{theta_str}\033[0m, r = \033[1m{r_str}\033[0m, C = \033[1m{C_str}\033[0m, S = \033[1m{S_str}\033[0m')

original: theta = 48.00, r = 79.90, C = 502.05, S = 66.94
case 1: theta = 13.93, r = 267.95, C = 1683.58, S = 65.16
case 2: theta = 27.87, r = 134.97, C = 848.05, S = 65.65
case 3: theta = 41.80, r = 91.10, C = 572.42, S = 66.46
case 4: theta = 55.73, r = 69.53, C = 436.88, S = 67.64
case 5: theta = 69.67, r = 56.90, C = 357.51, S = 69.18


## Functions

In [ ]:
# In this cell, I want to obtain the values of beta1, beta2, and N_cr useful for out-of-plane buckling analysis

# Define how the beta1 parameter should vary for out-of-plane buckling analysis
def close_to_outplane(y, low, high):
    """ Given an input between two extremes, returns the value of the nearest extreme """

    # Define the width of the neighborhoods of y
    middle_point_outplane = abs((low - high) / 2) + low

    # If y is greater than or equal to the middle_point value, it should return high; otherwise, low
    if y >= middle_point_outplane:
        return high
    else:
        return low

def get_beta1_and_Iy(case_data):
    f = case_data.get('f', None)
    Iy = case_data.get('Iy', None)
    if f is not None and Iy is not None:
        f_ratio = f / L

        ratio_to_beta1 = {
            0.05: 0.50,
            0.10: 0.54,
            0.20: 0.65,
            0.30: 0.82,
            0.40: 1.07,
        }

        if f_ratio <= 0.05:
            return ratio_to_beta1[0.05], Iy
        elif f_ratio >= 0.40:
            return ratio_to_beta1[0.40], Iy
        elif f_ratio < 0.10:
            return ratio_to_beta1[close_to_outplane(f_ratio, 0.05, 0.10)], Iy
        elif f_ratio < 0.20:
            return ratio_to_beta1[close_to_outplane(f_ratio, 0.10, 0.20)], Iy
        elif f_ratio < 0.30:
            return ratio_to_beta1[close_to_outplane(f_ratio, 0.20, 0.30)], Iy
        elif f_ratio < 0.40:
            return ratio_to_beta1[close_to_outplane(f_ratio, 0.30, 0.40)], Iy
    else:
        return None, None

# Print the values of beta1, beta2, and N_cr
for case, study_data in studies.items():
    beta1, Iy = get_beta1_and_Iy(study_data)
    if beta1 is not None and Iy is not None:
        beta2 = 0.65
        N_cr = (3.14 / (beta1 * beta2 * (S / 2))) ** 2 * E * Iy
        N_cr = round(N_cr)  # Round to the nearest integer

        # Convert numerical values to strings and format in bold
        beta1_str = "{:.2f}".format(beta1)
        beta2_str = "{:.2f}".format(beta2)
        N_cr_str = str(N_cr)

        print(f'{case}: beta1 = \033[1m{beta1_str}\033[0m, beta2 = \033[1m{beta2_str}\033[0m, N_cr = \033[1m{N_cr_str}\033[0m')

original: beta1 = 0.54, beta2 = 0.65, N_cr = 325840
case 1: beta1 = 0.50, beta2 = 0.65, N_cr = 525859
case 2: beta1 = 0.50, beta2 = 0.65, N_cr = 434120
case 3: beta1 = 0.54, beta2 = 0.65, N_cr = 353930
case 4: beta1 = 0.54, beta2 = 0.65, N_cr = 300559
case 5: beta1 = 0.65, beta2 = 0.65, N_cr = 284987


In [ ]:
# In this cell, I want to obtain the value of K for each design scenario

# Define how the K parameter should vary for snap-through analysis
def close_to(x, low, high):
    """ Given an input between two extremes, returns the value of the nearest extreme """

    # Define the width of the neighborhoods of x
    middle_point = abs((low - high) / 2) + low

    # If x is greater than or equal to the middle_point value, it should return high; otherwise, low
    if x >= middle_point:
        return high
    else:
        return low


def get_K(case_data):
    """ Given a case data dictionary, returns K """
    f = case_data.get('f', None)
    if f is not None:
        # Define the value of f_ratio
        f_ratio = f / L

        # Define a dictionary for K values
        # I have indicated K1 for values that K can take in the case of a double-articulated arch
        # I have indicated K2 for values that K can take in the case of a non-articulated arch
        ratio_to_K = {
            0.05: {'K1': 35, 'K2': 319},
            0.075: {'K1': 23, 'K2': 97},
            0.10: {'K1': 17, 'K2': 42},
            0.15: {'K1': 10, 'K2': 13},
            0.2: {'K1': 8, 'K2': 6},
        }

        # Determine the type of K to use based on f
        # From the conducted experiments, I have seen that for f <= 4, it is very difficult to handle K2. Therefore, in this case, set K1, otherwise K2
        K_type = 'K1' if f <= 4 else 'K2'

        if f_ratio in ratio_to_K:
            return ratio_to_K[f_ratio][K_type]
        else:
            # If f_ratio is not exactly in ratio_to_K, approximate to the nearest values
            closest_ratio = min(ratio_to_K.keys(), key=lambda x: abs(x - f_ratio))
            return ratio_to_K[closest_ratio][K_type]

    return None

# Print the values of K for each design scenario
for case, study_data in studies.items():
    K = get_K(study_data)
    if K is not None:
        # Convert the numerical value of K to a string and format in bold
        K_str = str(K)
        print(f'{case}: K = \033[1m{K_str}\033[0m')

original: K = 42
case 1: K = 35
case 2: K = 35
case 3: K = 42
case 4: K = 42
case 5: K = 13


In [ ]:
# In this cell, I calculate the objective function, i.e., the amount of steel required to build the bridge arch and the corresponding cost and CO2 emission values

# Define the function to be optimized, i.e., the objective function
def get_CO2eqe(study_data):
    Aa, f, Iy, Ix, N_Eda, N_Edi, M_Edi, V_Edi, alpha_cr = (
        study_data['Aa'], study_data['f'], study_data['Iy'], study_data['Ix'],
        study_data['N_Eda'], study_data['N_Edi'],
        study_data['M_Edi'], study_data['V_Edi'], study_data['alpha_cr']
    )

    # Within the objective function, define the constraints to which it will be subject

    # Constraint: Calculate N_Rda (resistant compression stress N of the arch)
    N_Rda = (fyk / 1.1) * Aa
    #"N_Rda should be compared with the N_Eda stress provided by Straus, which varies based on the geometry adopted by the arch, if I had the API"

    # Constraint: Calculate N_Rdi (resistant tension stress N of the deck)
    N_Rdi = (fyk / 1.1) * Ai
    #"N_Rdi is a constant because the deck section always remains the same"
    #"N_Rdi should be compared with the N_Edi stress provided by Straus, which varies based on the geometry adopted by the arch, if I had the API"

    # Constraint: Calculate M_Rdi (resistant bending moment M of the deck)
    M_Rdi = (W * fyk) / 1.1
    #"M_Rdi is a constant because the deck section always remains the same"
    #"M_Rdi should be compared with the M_Edi stress provided by Straus, which varies based on the geometry adopted by the arch, if I had the API"

    # Constraint: Calculate V_Rdi (resistant shear stress V of the deck)
    V_Rdi = (fyk / (1.1 * sqrt(3))) * Ai
    #"V_Rdi is a constant because the deck section always remains the same"
    #"V_Rdi should be compared with the V_Edi stress provided by Straus, which varies based on the geometry adopted by the arch, if I had the API"

    # Constraint: Snap-through
    K = get_K(study_data) # Calculate K based on f
    tmp = L * sqrt((E * Aa) / (12 * E * Ix))
    #"tmp is the value to be greater than K to satisfy the snap-through analysis"



    # Verify the constraints
    if N_Rda <= N_Eda or N_Rdi <= N_Edi or M_Rdi <= M_Edi or V_Rdi <= V_Edi or N_cr <= N_Eda or tmp < K:
        penalty = 1e9  # Set a high penalty if the constraints are not satisfied
        weight_tot = 1e9
        cost_tot = 1e9
        CO2eqe = penalty
        # Print "Penalty" if the penalty is activated
        if N_Rda <= N_Eda:
            print("Penalty N_Rda <= N_Eda")
        if N_Rdi <= N_Edi:
            print("Penalty N_Rdi <= N_Edi")
        if M_Rdi <= M_Edi:
            print("Penalty M_Rdi <= M_Edi")
        if V_Rdi <= V_Edi:
            print("Penalty V_Rdi <= V_Edi")
        if N_cr <= N_Eda:
            print("Penalty N_cr <= N_Eda")
        if tmp < K:
            print("Penalty tmp < K")

    else:
        # Calculate the value to be minimized

        theta = (f * 48) / 6.89  # [°]
        r = (L / 2) * (1 / math.sin(math.radians(theta / 2)))  # [m]
        C = 2 * math.pi * r  # [m]
        S = (C * theta) / 360  # [m]

        weight_tot = rho * Aa * S  # [kg]
        cost_tot = rho * Aa * costS355 * S  # [euros]
        CO2eqe = rho * Aa * GWP * S  # [kg]

    return "{:.0f}".format(weight_tot), "{:.0f}".format(cost_tot), "{:.0f}".format(CO2eqe)



# Procedure to print Weight, Cost, and CO2eqe for each analyzed case
for case, study_data in studies.items():
    beta1, Iy = get_beta1_and_Iy(study_data)
    if beta1 is not None and Iy is not None:
        beta2 = 0.65
        N_cr = (3.14 / (beta1 * beta2 * (S / 2))) ** 2 * E * Iy

        # Call get_CO2eqe with appropriate values
        weight_tot, cost_tot, CO2eqe = get_CO2eqe(study_data)

        # Convert numerical values to strings and format in bold
        weight_tot_str = str(weight_tot)
        cost_tot_str = str(cost_tot)
        CO2eqe_str = str(CO2eqe)

        print(f'{case}: Weight = \033[1m{weight_tot_str}\033[0m, Cost = \033[1m{cost_tot_str}\033[0m, CO2eqe = \033[1m{CO2eqe_str}\033[0m')

original: Weight = 85654, Cost = 61671, CO2eqe = 175076
case 1: Weight = 86445, Cost = 62240, CO2eqe = 176694
case 2: Weight = 82966, Cost = 59735, CO2eqe = 169582
case 3: Weight = 79827, Cost = 57475, CO2eqe = 163166
case 4: Weight = 82826, Cost = 59635, CO2eqe = 169296
case 5: Weight = 84722, Cost = 61000, CO2eqe = 173173


In [ ]:
# In this cell, I want to obtain the phrases "Second order analysis required" or "Second order analysis not required" based on the value of alpha_cr for each design scenario

def buckling(study):
    alpha_cr = study.get('alpha_cr', None)

    if alpha_cr is not None and alpha_cr >= 10:
        return '\033[1mSecond order analysis not required\033[0m'
    else:
        return '\033[1mSecond order analysis required\033[0m'

# Iterate over each case in the studies dictionary
for case, study_data in studies.items():
    result = buckling(study_data)

    # Print the result with proper formatting
    print(f'{case}: {result}')

original: Second order analysis required
case 1: Second order analysis required
case 2: Second order analysis required
case 3: Second order analysis not required
case 4: Second order analysis not required
case 5: Second order analysis not required


In [ ]:
# This cell is only for ensuring that, in the final results at the end of the process, the check for the second-order analysis is printed in bold

def buckling(study):
    if isinstance(study, dict):
        alpha_cr = study.get('alpha_cr', None)
    else:
        alpha_cr = study

    if alpha_cr is not None and alpha_cr >= 10:
        return '\033[1mSecond order analysis not required\033[0m'
    else:
        return '\033[1mSecond order analysis required\033[0m'

In [ ]:
# In this cell, I want to obtain the final results of the snap-through check (Positive or Negative check) and the corresponding value of K for each design scenario

for case, study_data in studies.items():
    # Call get_K with the appropriate values
    K = get_K(study_data)
    tmp = L * sqrt((E * study_data['Aa']) / (12 * E * study_data['Ix']))

    # Snap-through check
    if tmp > K:
        snap_result = f'\033[1mPositive check ({K})\033[0m'
    else:
        snap_result = f'\033[1mNegative check ({K})\033[0m'

    # Print the Snap result
    print(f'{case}: {snap_result}')

original: Positive check (42)
case 1: Positive check (35)
case 2: Positive check (35)
case 3: Positive check (42)
case 4: Positive check (42)
case 5: Positive check (13)


In [ ]:
# In this cell, I perform strength verifications

# Strength check for arch compression
def arch_compression_check(N_Rda, N_Eda):
    return '\033[1m==> Positive check\033[0m' if N_Rda >= N_Eda else '\033[1m==> Negative check\033[0m'

# Strength check for deck tension
def deck_tension_check(N_Rdi, N_Edi):
    return '\033[1m==> Positive check\033[0m' if N_Rdi >= N_Edi else '\033[1m==> Negative check\033[0m'

# Strength check for deck shear
def deck_shear_check(V_Rdi, V_Edi):
    return '\033[1m==> Positive check\033[0m' if V_Rdi >= V_Edi else '\033[1m==> Negative check\033[0m'

# Strength check for deck bending moment
def deck_bendingmoment_check(M_Rdi, M_Edi):
    return '\033[1m==> Positive check\033[0m' if M_Rdi >= M_Edi else '\033[1m==> Negative check\033[0m'

# Snap-through check
def snap_through_check(tmp, K):
    return '\033[1mPositive check\033[0m' if tmp > K else '\033[1mNegative check\033[0m'

# Iteration for each case
for case, study_data in studies.items():
    beta1, Iy = get_beta1_and_Iy(study_data)
    if beta1 is not None and Iy is not None:
        beta2 = 0.65
        N_cr = (3.14 / (beta1 * beta2 * (S / 2))) ** 2 * E * Iy

        # Call get_CO2eqe with the appropriate values
        weight_tot, cost_tot, CO2eqe = get_CO2eqe(study_data)

        # Call snap_through_check with the appropriate values
        K = get_K(study_data)
        tmp = L * sqrt((E * study_data['Aa']) / (12 * E * study_data['Ix']))
        snap_result = snap_through_check(tmp, K)

        # Strength verifications
        arch_comp_result = arch_compression_check((fyk / 1.1) * study_data['Aa'], study_data['N_Eda'])
        deck_tension_result = deck_tension_check((fyk / 1.1) * Ai, study_data['N_Edi'])
        deck_shear_result = deck_shear_check((fyk / (1.1 * sqrt(3))) * Ai, study_data['V_Edi'])
        deck_bending_result = deck_bendingmoment_check((W * fyk) / 1.1, study_data['M_Edi'])

        # Print the strength verification results in bold
        print(f'{case}: Arch Compression {arch_comp_result}, Deck Tension {deck_tension_result}, Deck Shear {deck_shear_result}, Deck Bending Moment {deck_bending_result}, Snap Through {snap_result}')

original: Arch Compression ==> Positive check, Deck Tension ==> Positive check, Deck Shear ==> Positive check, Deck Bending Moment ==> Positive check, Snap Through Positive check
case 1: Arch Compression ==> Positive check, Deck Tension ==> Positive check, Deck Shear ==> Positive check, Deck Bending Moment ==> Positive check, Snap Through Positive check
case 2: Arch Compression ==> Positive check, Deck Tension ==> Positive check, Deck Shear ==> Positive check, Deck Bending Moment ==> Positive check, Snap Through Positive check
case 3: Arch Compression ==> Positive check, Deck Tension ==> Positive check, Deck Shear ==> Positive check, Deck Bending Moment ==> Positive check, Snap Through Positive check
case 4: Arch Compression ==> Positive check, Deck Tension ==> Positive check, Deck Shear ==> Positive check, Deck Bending Moment ==> Positive check, Snap Through Positive check
case 5: Arch Compression ==> Positive check, Deck Tension ==> Positive check, Deck Shear ==> Positive check, Deck

## Final results

In [ ]:
# In this cell, I want to summarize all the results of interest found for each design scenario

def calculate_resistance_values(Aa):
    N_Rda = (fyk / 1.1) * Aa
    N_Rdi = (fyk / 1.1) * Ai
    M_Rdi = (W * fyk) / 1.1
    V_Rdi = (fyk / (1.1 * math.sqrt(3))) * Ai
    return N_Rda, N_Rdi, M_Rdi, V_Rdi

def print_results(Aa, f, Iy, Ix, N_Eda, N_Edi, M_Edi, V_Edi, alpha_cr, snap_result, N_Rda, N_Rdi, M_Rdi, V_Rdi, S):
    print(f"Design variable Aa: \033[1m{Aa} m^2\033[0m")
    print("Design variable f: \033[1m{:.2f} m\033[0m".format(f))
    print(f"Dependent variable Iy: \033[1m{Iy} m^4\033[0m")
    print(f"Dependent variable Ix: \033[1m{Ix} m^4\033[0m")

    # Print the first row of results below the "Dependent variable Ix" row
    if result is not None:
        theta, r, C, S = result
        # Round values to two decimal places
        theta = round(theta, 2)
        r = round(r, 2)
        C = round(C, 2)
        S = round(S, 2)

        # Print numerical values in bold
        print(f'Design variables: theta = \033[1m{theta}\033[0m, r = \033[1m{r}\033[0m, C = \033[1m{C}\033[0m, S = \033[1m{S}\033[0m')

    print("Comparison N_Eda & N_Rda: \033[1m{:.2f} kN, {:.2f} kN\033[0m".format(N_Eda, N_Rda), arch_compression_check(N_Rda, N_Eda))
    print("Comparison N_Edi & N_Rdi: \033[1m{:.2f} kN, {:.2f} kN\033[0m".format(N_Edi, N_Rdi), deck_tension_check(N_Rdi, N_Edi))
    print("Comparison V_Edi & V_Rdi: \033[1m{:.2f} kN, {:.2f} kN\033[0m".format(V_Edi, V_Rdi), deck_shear_check(V_Rdi, V_Edi))
    print("Comparison M_Edi & M_Rdi: \033[1m{:.2f} kN m, {:.2f} kN m\033[0m".format(M_Edi, M_Rdi), deck_bendingmoment_check(M_Rdi, M_Edi))
    print("Buckling check:", buckling(alpha_cr))
    print("Snap-through analysis:", snap_result)
    tmp = get_CO2eqe(study_data)
    print("Objective function (weight_tot [kg], cost_tot [€], CO2eqe [kg]):", tmp)
    val_0 = int(tmp[0])
    val_1 = int(tmp[1])
    val_2 = int(tmp[2])
    if val_2 < 175076:
        print(f"- kg of steel saved: \033[1m{abs(val_0 - 85654)} kg\033[0m;")
        print(f"- Cost saved: \033[1m{abs(val_1 - 61671)} €\033[0m;")
        print(f"- kg of CO2 emitted saved: \033[1m{abs(val_2 - 175076)} kg\033[0m;")
        print(f'Savings: \033[1m{100 * abs(val_2 - 175076) / 175076:.2f} %\033[0m')
    else:
        print(f'No savings; surplus: \033[1m{100 * abs(val_2 - 175076) / 175076:.2f} %\033[0m')
        print(f"- kg of steel exceeded: \033[1m{abs(val_0 - 85654)} kg\033[0m;")
        print(f"- Cost exceeded: \033[1m{abs(val_1 - 61671)} €\033[0m;")
        print(f"- kg of CO2 emitted exceeded: \033[1m{abs(val_2 - 175076)} kg\033[0m;")

# Iteration over all study cases
for case, study_data in studies.items():
    # Call calculate_resistance_values with the appropriate values
    N_Rda, N_Rdi, M_Rdi, V_Rdi = calculate_resistance_values(study_data['Aa'])
    result = calculate_S(study_data)
    if result is not None:
        theta, r, C, S = result
        # Round values to two decimal places
        theta = round(theta, 2)
        r = round(r, 2)
        C = round(C, 2)
        S = round(S, 2)

    # Call snap_through_check with the appropriate values
    K = get_K(study_data) # Calculate K based on f
    tmp = L * math.sqrt((E * study_data['Aa']) / (12 * E * study_data['Ix']))
    snap_result = snap_through_check(tmp, K)

    # Snap-through verification
    if tmp > K:
        snap_result = f'\033[1mPositive check ({K})\033[0m'
    else:
        snap_result = f'\033[1mNegative check ({K})\033[0m'

    # Pass all necessary variables as arguments when calling print_results
    print_results(snap_result=snap_result, N_Rda=N_Rda, N_Rdi=N_Rdi, M_Rdi=M_Rdi, V_Rdi=V_Rdi, S=S, **study_data)

    print("-" * 100) # This command is only for adding the dashed line that separates the various analyzed study cases (specifically, 100 dashes)

Design variable Aa: 0.163 m^2
Design variable f: 6.89 m
Dependent variable Iy: 0.0232 m^4
Dependent variable Ix: 0.00769 m^4
Design variables: theta = 48.0, r = 79.9, C = 502.05, S = 66.94
Comparison N_Eda & N_Rda: 12498.00 kN, 52604.55 kN ==> Positive check
Comparison N_Edi & N_Rdi: 11810.00 kN, 199445.45 kN ==> Positive check
Comparison V_Edi & V_Rdi: 2000.00 kN, 115149.89 kN ==> Positive check
Comparison M_Edi & M_Rdi: 11911.00 kN m, 211709.09 kN m ==> Positive check
Buckling check: Second order analysis required
Snap-through analysis: Positive check (42)
Objective function (weight_tot [kg], cost_tot [€], CO2eqe [kg]): ('85654', '61671', '175076')
No savings; surplus: 0.00 %
- kg of steel exceeded: 0 kg;
- Cost exceeded: 0 €;
- kg of CO2 emitted exceeded: 0 kg;
----------------------------------------------------------------------------------------------------
Design variable Aa: 0.169 m^2
Design variable f: 2.00 m
Dependent variable Iy: 0.0321 m^4
Dependent variable Ix: 0.00513 m^4